In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from model import CardClassifier

train_path = "/content/drive/MyDrive/playing_cards/train" #/content/drive/MyDrive/playing_cards/train, ../../playing_cards/train
valid_path = "/content/drive/MyDrive/playing_cards/valid" #/content/drive/MyDrive/playing_cards/valid, ../../playing_cards/valid
test_path = "/content/drive/MyDrive/playing_cards/test" #/content/drive/MyDrive/playing_cards/test, ../../playing_cards/test

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [6]:
train_dataset = datasets.ImageFolder(train_path, transform=transform)
valid_dataset = datasets.ImageFolder(valid_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

# load data into the GPU memory
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, pin_memory=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
model = CardClassifier().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
num_epochs = 100  # Set the number of training epochs

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        # Move inputs and labels to GPU
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average training loss for the epoch
    average_train_loss = running_loss / len(train_loader)

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in valid_loader:
            # Move inputs and labels to GPU
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    # Calculate average validation loss and accuracy for the epoch
    average_val_loss = val_loss / len(valid_loader)
    accuracy = correct / total

    print(f"Epoch {epoch+1}/{num_epochs} => "
          f"Train Loss: {average_train_loss:.4f}, "
          f"Validation Loss: {average_val_loss:.4f}, "
          f"Validation Accuracy: {accuracy * 100:.2f}%")


Epoch 1/100 => Train Loss: 0.0223, Validation Loss: 1.0249, Validation Accuracy: 80.38%
Epoch 2/100 => Train Loss: 0.0215, Validation Loss: 1.0218, Validation Accuracy: 80.38%
Epoch 3/100 => Train Loss: 0.0242, Validation Loss: 1.1586, Validation Accuracy: 82.26%
Epoch 4/100 => Train Loss: 0.0335, Validation Loss: 1.3834, Validation Accuracy: 78.49%
Epoch 5/100 => Train Loss: 0.0634, Validation Loss: 1.3870, Validation Accuracy: 76.23%
Epoch 6/100 => Train Loss: 0.0805, Validation Loss: 1.2808, Validation Accuracy: 77.74%
Epoch 7/100 => Train Loss: 0.0457, Validation Loss: 1.3491, Validation Accuracy: 76.98%
Epoch 8/100 => Train Loss: 0.0378, Validation Loss: 1.3373, Validation Accuracy: 75.09%
Epoch 9/100 => Train Loss: 0.0227, Validation Loss: 1.4774, Validation Accuracy: 80.00%
Epoch 10/100 => Train Loss: 0.0099, Validation Loss: 1.4543, Validation Accuracy: 78.87%
Epoch 11/100 => Train Loss: 0.0130, Validation Loss: 1.4112, Validation Accuracy: 79.25%
Epoch 12/100 => Train Loss: 0.

In [11]:
torch.save(model.state_dict(), "card_classifier.pth")